## Part 1

In [33]:
#import pandas
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
import numpy as np
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [4]:
# Import and use Beautiful Soup to get the data from the site
import requests
from bs4 import BeautifulSoup
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
src = result.content

In [5]:
#Get the data from the website
soup = BeautifulSoup(src, 'lxml')
data = []

#Find all the tables, relevant data
for td_tag in soup.find_all("td"):
    data.append(td_tag.text.strip())

#Create lists to store sepearte informaton
zip_codes = []
boroughs = []
neighberhoods = []

#Go through data and split it in to categories, plus clean it up
for x in data:
    zip_codes.append(x[0:3])  
    boroughs.append(x[3:].replace(")",'').split('(')[0])
    try:
        ng = x[3:].replace(")",'').replace(' /', ',').split('(')[1]
        neighberhoods.append(ng)
    except:
        neighberhoods.append('Not assigned')


#make a dictionary from lists
d = {'Postal Code': zip_codes, 'Borough': boroughs, 'Neighborhood': neighberhoods}
#convert to dataframe
df_t = pd.DataFrame(d)

#clean up datafram

df = df_t[df_t['Borough'] != 'Not assigned']
df = df[df['Postal Code'].map(len) > 2]
df = df[df['Postal Code'].str.isalnum()]

#reset index after cleanup, and get rid of old index
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)
df.head()


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
#shape of the dataframe
df.shape

(103, 3)

## Part 2

In [7]:
#import coordinational data from CSV
cords = pd.read_csv('Geospatial_Coordinates.csv')
pd.DataFrame(cords)

#Merge the two data frames together, to have the Latitude and Logitude
df_all = pd.merge(df, cords, on=['Postal Code'])
df_all.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Part 3
Analyze Each Neoghborhood

In [8]:
#fidn the locationas for city of Toronto
city = 'Toronto, ON'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(city)
lat = location.latitude
lng = location.longitude
print('Geo Location is : {} Latitiude, {} Longitude'. format(lat, lng))

Geo Location is : 43.6534817 Latitiude, -79.3839347 Longitude


In [9]:
#Map of toronto
map_toronto = folium.Map(location=[lat,lng], zoom_start=12)
map_toronto

In [10]:
for lat, lng, bor, neigh in zip(df_all['Latitude'], df_all['Longitude'], df_all['Borough'], df_all['Neighborhood']):

    label= '{}, {}'.format(neigh, bor)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        popup=label,
        radius=5,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

## Boroughs that only contain Toronto

In [11]:
df_tor_bor = df_all[df_all['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_tor_bor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [12]:

#Map of toronto
map_toronto_center = folium.Map(location=[lat,lng], zoom_start=12)

#plot on a new map
for lat, lng, bor, neigh in zip(df_tor_bor['Latitude'], df_tor_bor['Longitude'], df_tor_bor['Borough'], df_tor_bor['Neighborhood']):

    label= '{}, {}'.format(neigh, bor)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        popup=label,
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_center)

map_toronto_center

In [13]:
#Forsquare
CLIENT_ID = 'EZH4ERBYQUEFWGXF300Z2F2MGAVYER22QXQL50PBV4PTO02B' # your Foursquare ID
CLIENT_SECRET = 'EJV41SURWZJNTRURBKEPEXQRT0JWWAOELP50MPUJHKBQSSBL' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value



In [14]:
#Lets explore nearby vneues for a single Neighborhood
neigh_name = df_tor_bor.loc[0,'Neighborhood']
neigh_lat = df_tor_bor.loc[0, 'Latitude']
neigh_lng = df_tor_bor.loc[0, 'Longitude']
print('{}, Lat:{}, Long:{}'. format(neigh_name, neigh_lat, neigh_lng))

Regent Park, Harbourfront, Lat:43.6542599, Long:-79.3606359


In [15]:
#Get results for that neighborhood
LIMIT = 100 #limit number of venues
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
results = requests.get(url).json()


In [16]:
#Extract category of venues
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)

#filter columns
filtered_columns = ['venue.name','venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues[['venue.categories', 'venue.location.lat', 'venue.location.lng']]

#filter the categories
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,categories,lat,lng
0,Brewery,43.663483,-79.319824
1,Farmers Market,43.664901,-79.319784
2,Comic Shop,43.664484,-79.325162
3,Pizza Place,43.664685,-79.324164
4,Burrito Place,43.664653,-79.325584


### Get Nearby Venues for the 'Toronto' neighborhoods


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names = df_all['Neighborhood'], latitudes=df_all['Latitude'], longitudes=df_all['Longitude'])

In [20]:
# how many venues and count
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,57,57,57,57,57,57
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16


In [21]:
#How many unique categroies
len(toronto_venues['Venue Category'].unique())

270

## Analysis

In [22]:
#one hot encoding
tor_venues_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep="")

#add neighborhoods columns back to dataframe
tor_venues_onehot['Neighborhood'] = toronto_venues['Neighborhood']

#move the neighborhood column forward
fixed_neighborhood = [tor_venues_onehot.columns[-1]] + list(tor_venues_onehot.columns[:-1])
tor_venues_onehot = tor_venues_onehot[fixed_neighborhood]

tor_venues_onehot.head()



,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
tor_venues_grouped = tor_venues_onehot.groupby('Neighborhood').mean().reset_index()
tor_venues_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [56]:
#Most common venue in each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
num_top_venues = 10

columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{} Most Common Venue'.format(ind+1))
    except:
        columns.append('{} Most Common Venue'.format(ind+1))

#create new dataframe
neighberhoods_venues_sort = pd.DataFrame(columns= columns)
neighberhoods_venues_sort['Neighborhood'] = tor_venues_grouped['Neighborhood']

for ind in np.arange(tor_venues_grouped.shape[0]):
    neighberhoods_venues_sort.iloc[ind, 1:] = return_most_common_venues(tor_venues_grouped.iloc[ind,:], num_top_venues)

neighberhoods_venues_sort.head()


,Neighborhood,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Pharmacy,Pub,Dance Studio,Women's Store,Diner,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Pharmacy,Deli / Bodega,Supermarket,Intersection,Sushi Restaurant,Shopping Mall,Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Juice Bar,Restaurant,Pizza Place,Thai Restaurant,Pharmacy,Sushi Restaurant


## Cluster the neighborhoods

In [58]:
#Number of clusters
k = 5

tor_venues_grouped_cluster = tor_venues_grouped.drop('Neighborhood',1)
#K means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(tor_venues_grouped_cluster)
#check cluster label generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [59]:
#add clustering labels
neighberhoods_venues_sort.insert(0, 'Cluster Labels', kmeans.labels_)

tor_venues_merged = df_tor_bor.join(neighberhoods_venues_sort.set_index('Neighborhood'), on='Neighborhood')

tor_venues_merged.head()



,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Gym / Fitness Center,Event Space,Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Bubble Tea Shop,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Movie Theater,Lingerie Store
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cosmetics Shop,Cocktail Bar,Restaurant,Gastropub,Farmers Market,Beer Bar,Italian Restaurant,Japanese Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Bakery,Cocktail Bar,Coffee Shop,Seafood Restaurant,Pub,Restaurant,Pharmacy,Beer Bar,Farmers Market,Cheese Shop


In [60]:
# Visualize the clusters
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

#color schemes of clusters
x = np.arange(k)
ys = [i + x +(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0,1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#markers on the map
markers_colors = []
for lat, lng, ngh, cluster in zip(
    tor_venues_merged['Latitude'],
    tor_venues_merged['Longitude'],
    tor_venues_merged['Neighborhood'],
    tor_venues_merged['Cluster Labels']):
    label = folium.Popup(str(ngh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup=label,
        color = rainbow[cluster-1],
        fill= True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7
    ).add_to(map_clusters)

map_clusters






In [61]:
#cluster 1
tor_venues_merged.loc[tor_venues_merged['Cluster Labels'] == 0, tor_venues_merged.columns[[1] + list(range(5, tor_venues_merged.shape[1]))]]


,Borough,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Gym / Fitness Center,Event Space,Restaurant
1,Downtown Toronto,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Bubble Tea Shop,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Movie Theater,Lingerie Store
2,Downtown Toronto,0,Café,Coffee Shop,Cosmetics Shop,Cocktail Bar,Restaurant,Gastropub,Farmers Market,Beer Bar,Italian Restaurant,Japanese Restaurant
3,East Toronto,0,Trail,Pub,Health Food Store,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,Downtown Toronto,0,Bakery,Cocktail Bar,Coffee Shop,Seafood Restaurant,Pub,Restaurant,Pharmacy,Beer Bar,Farmers Market,Cheese Shop
5,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Restaurant,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Miscellaneous Shop
6,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Nightclub,Candy Store,Coffee Shop,Restaurant,Dessert Shop
7,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Thai Restaurant,Deli / Bodega,Clothing Store,Gym,Hotel,Bar,Steakhouse
8,West Toronto,0,Bakery,Pharmacy,Middle Eastern Restaurant,Pet Store,Café,Bank,Bar,Portuguese Restaurant,Supermarket,Park
10,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Italian Restaurant,Restaurant,Brewery,Scenic Lookout,Park


In [62]:
#cluster 2
tor_venues_merged.loc[tor_venues_merged['Cluster Labels'] == 1, tor_venues_merged.columns[[1] + list(range(5, tor_venues_merged.shape[1]))]]


,Borough,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue


In [63]:
#cluster 3
tor_venues_merged.loc[tor_venues_merged['Cluster Labels'] == 2, tor_venues_merged.columns[[1] + list(range(5, tor_venues_merged.shape[1]))]]


,Borough,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
9,East YorkEast Toronto,2,Park,Convenience Store,Metro Station,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Department Store
18,Central Toronto,2,Park,Swim School,Bus Line,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega,Distribution Center
33,Downtown Toronto,2,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


In [64]:
#cluster 4
tor_venues_merged.loc[tor_venues_merged['Cluster Labels'] == 3, tor_venues_merged.columns[[1] + list(range(5, tor_venues_merged.shape[1]))]]


,Borough,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
29,Central Toronto,3,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop


In [65]:
#cluster 5
tor_venues_merged.loc[tor_venues_merged['Cluster Labels'] == 4, tor_venues_merged.columns[[1] + list(range(5, tor_venues_merged.shape[1]))]]


,Borough,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
